In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#@title **INSTALLS**
!pip install -q deepface
!pip install -q face_recognition
!pip install -q mediapipe
!pip install -q opencv-python
!git clone https://github.com/foivospar/Arc2Face.git
#!pip install -r /content/Arc2Face/requirements.txt
!wget -q -O detector.tflite -q https://storage.googleapis.com/mediapipe-models/face_detector/blaze_face_short_range/float16/1/blaze_face_short_range.tflite


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.4/88.4 kB 6.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.6/108.6 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 61.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.1/100.1 MB 21.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.9/35.9 MB 63.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 30.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-metadata 1.15.0 requires protobuf<4.21,>=3.20.3; python_version < "3.11", but you have protobuf 4.25.4 which is incompatible.
Cloning into 'Arc2Face'...
remote: Enumerating 

In [ ]:
!pip install numpy<1.24.0
!pip install torch==2.0.1
!pip install torchvision==0.15.2
!pip install diffusers==0.23.0
!pip install peft
!pip install accelerate
!pip install insightface
!pip install onnxruntime-gpu
!pip install gradio
!pip install transformers==4.33.0

/bin/bash: line 1: 1.24.0: No such file or directory
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 116.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 95.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 57.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.4/168.4 MB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 MB 40.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.6/102.6 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.2/173.2 MB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.1/177.1 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.6/98.6 kB 9.0 MB/s e

In [ ]:
#@title **IMPORT REQS**
!mkdir -p ./models/antelopev2
#!mkdir -p ./search_images
#!mkdir -p ./search_videos

import pickle
import os
import pandas as pd
import numpy as np
import json
import logging
from tqdm.notebook import tqdm
import cv2
import math
import shutil
from sklearn.metrics.pairwise import cosine_similarity
from PIL import Image
import matplotlib.pyplot as plt

import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision
from typing import Tuple, Union

import sys
sys.path.append('/content/Arc2Face')

import insightface
from insightface.app import FaceAnalysis
from arc2face import CLIPTextModelWrapper, project_face_embs
import torch
from insightface.app.face_analysis import Face

from huggingface_hub import hf_hub_download

hf_hub_download(repo_id="FoivosPar/Arc2Face", filename="arc2face/config.json", local_dir="./models")
hf_hub_download(repo_id="FoivosPar/Arc2Face", filename="arc2face/diffusion_pytorch_model.safetensors", local_dir="./models")
hf_hub_download(repo_id="FoivosPar/Arc2Face", filename="encoder/config.json", local_dir="./models")
hf_hub_download(repo_id="FoivosPar/Arc2Face", filename="encoder/pytorch_model.bin", local_dir="./models")

hf_hub_download(repo_id="FoivosPar/Arc2Face", filename="arcface.onnx", local_dir="./models/antelopev2")

print("done")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


arc2face/config.json:   0%|          | 0.00/1.78k [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/3.44G [00:00<?, ?B/s]

encoder/config.json:   0%|          | 0.00/560 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/492M [00:00<?, ?B/s]

arcface.onnx:   0%|          | 0.00/261M [00:00<?, ?B/s]

done


In [ ]:
#@title **SET UP DICT**

!cp -r /content/drive/MyDrive/models/antelopev2/* /content/models/antelopev2


base_folder = "/content/drive/MyDrive/faces" #@param {type:"string"}
emb_file = base_folder + '/NYPD_profile_embeddings.pkl'
json_file = base_folder + '/NYPD_profile_dict.json'


json_df = pd.read_json(json_file)
json_df.reset_index()

json_df = json_df[pd.notna(json_df['source_link'])]

img_names = []
for index, row in json_df.iterrows():
    badge_number = row['badge_number'][1:] if pd.notna(row['badge_number']) and row['badge_number'] else 'unknown'
    if pd.notna(row['allegations']):
        allegations = int(row['allegations']) if isinstance(row['allegations'], float) else row['allegations']
    else:
        allegations = 'unknown'
    name = row['name'].replace(' ', '_').replace(',', '').replace('.', '') if pd.notna(row['name']) and row['name'] else 'unknown'
    img_name = f"{badge_number}_{allegations}_{name}.jpg"
    img_names.append(img_name)

json_df['img_name'] = img_names

with open(emb_file, 'rb') as f:
    emb_data = pickle.load(f)
    emb_data = list(zip(emb_data[0], emb_data[1]))
    emb_df = pd.DataFrame(emb_data, columns=['img_name', 'embedding'])


merged = pd.merge(json_df, emb_df, left_on='source_link', right_on='img_name')


In [ ]:
non_faces = ['#17032', '#10171', '#13404', '#16688', '#648', '#7320', '#13767', '#2126', '#5207', '#3061', '#13449', '#9322', '#19022', '#15035', '#6901', '#3001', '#4736', '#2543', '#11435', '#19708', '#12214', '#14247', '#13749', '#14629', '#7535', '#21593', '#3754', '#3667', '#86']

filtered = merged[~merged['badge_number'].isin(non_faces)]

print(len(filtered))

11649


In [ ]:
#@title **EMBED FUNCTIONS**

app = FaceAnalysis(name='antelopev2', root='./', providers=['CUDAExecutionProvider', 'CPUExecutionProvider'])
app.prepare(ctx_id=0, det_size=(128, 128))

def expand_and_pad_image(image, target_size=512):
    # Get the current size of the image
    h, w, _ = image.shape

    # Determine the scaling factor to resize the image
    scale_factor = target_size / max(h, w)

    # Resize the image
    new_w = int(w * scale_factor)
    new_h = int(h * scale_factor)
    resized_image = cv2.resize(image, (new_w, new_h))

    # Calculate padding values to center the image
    pad_w = (target_size - new_w) // 2
    pad_h = (target_size - new_h) // 2

    # Create a new image with padding
    padded_image = cv2.copyMakeBorder(
        resized_image,
        pad_h,
        target_size - new_h - pad_h,
        pad_w,
        target_size - new_w - pad_w,
        cv2.BORDER_CONSTANT,
        value=[0, 0, 0]
    )

    return padded_image

def convert_images_to_jpg(directory):
    for filename in os.listdir(directory):
        if filename.endswith(('.png', '.jpeg', '.bmp', '.gif', '.tiff')):
            img_path = os.path.join(directory, filename)
            img = Image.open(img_path)
            if img.mode in ('RGBA', 'LA'):
                img = img.convert('RGB')
            new_filename = os.path.splitext(filename)[0] + '.jpg'
            new_path = os.path.join(directory, new_filename)
            img.save(new_path, 'JPEG')
            os.remove(img_path)

            #print(f"Converted {filename} to {new_filename} and deleted the original file")


def generate_embedding(img):
    padded_img = expand_and_pad_image(img, target_size=512) #in BGR
    faces = app.get(padded_img)
    if faces:
        id_emb = torch.tensor(faces[0]['embedding'], dtype=torch.float32)[None].cuda()
        id_emb = id_emb / torch.norm(id_emb, dim=1, keepdim=True)
        id_emb = id_emb.cpu()
        return id_emb
    return None

def process_images_in_folder(folder_path):
    convert_images_to_jpg(folder_path)
    embeddings = []
    image_names = []
    all_image_files = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]

    for image_file in all_image_files:
      imgpath = os.path.join(folder_path, image_file)
      img = np.array(Image.open(imgpath))[:,:,::-1]

      embedding = generate_embedding(img)
      if embedding is not None:
            embeddings.append(embedding.numpy())
            image_names.append(image_file)
            print("Embedded " + image_file)
    search_data = list(zip(image_names, embeddings))
    search_df = pd.DataFrame(search_data, columns=['img_name', 'embedding'])
    return search_df


#search_df = process_images_in_folder()


Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: ./models/antelopev2/1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: ./models/antelopev2/2d106det.onnx landmark_2d_106 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: ./models/antelopev2/arcface.onnx recognition ['None', 3, 112, 112] 127.5 127.5
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: ./models/antelopev2/genderage.onnx genderage ['None', 3, 96, 96] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: ./models/antelopev2/scrfd_10g_bnkps.onnx detection [1, 3, '?', '?'] 127.5 128.0
set det-size: (128, 128)


In [ ]:
#@title **SEARCH FUNCTION SETUP**

def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

def find_top_k_similar_embeddings(search_embedding, df, k=10):
    similarities = []

    # Convert search_embedding to the correct shape for cosine_similarity
    #search_embedding = np.array(search_embedding)#.reshape(1, -1)

    for index, row in df.iterrows():
        print(search_embedding)
        other_embedding = row['embedding'][0]#.reshape(1, -1)
        similarity = cosine_similarity(search_embedding, other_embedding)#[0][0]
        similarities.append((row['name'], row['badge_number'], similarity, row['img_name']))

    # Sort by similarity in descending order and get top k
    similarities.sort(key=lambda x: x[2], reverse=True)
    top_k_similarities = similarities[:k]

    return top_k_similarities


In [ ]:
#@title **MP FACE DETECT**

BaseOptions = mp.tasks.BaseOptions
FaceDetector = mp.tasks.vision.FaceDetector
FaceDetectorOptions = mp.tasks.vision.FaceDetectorOptions
VisionRunningMode = mp.tasks.vision.RunningMode

def initialize_mediapipe_detector(model_path='detector.tflite', min_detection_confidence=0.5):
    options = FaceDetectorOptions(
        base_options=BaseOptions(model_asset_path=model_path),
        running_mode=VisionRunningMode.IMAGE,
        min_detection_confidence=min_detection_confidence
    )
    detector = FaceDetector.create_from_options(options)
    return detector


def _normalized_to_pixel_coordinates(normalized_x, normalized_y, image_width, image_height):
    x_px = min(math.floor(normalized_x * image_width), image_width - 1)
    y_px = min(math.floor(normalized_y * image_height), image_height - 1)
    return (x_px, y_px)


def detect_faces_mediapipe(detector, image_path):
    img = cv2.imread(image_path)
    height, width, _ = img.shape
    mp_image = mp.Image.create_from_file(image_path)
    detection_result = detector.detect(mp_image)
    face_data = []

    for detection in detection_result.detections:
        bbox = detection.bounding_box
        center_x = bbox.origin_x + bbox.width / 2
        center_y = bbox.origin_y + bbox.height / 2

        # Expand the width and height by 1.1x
        new_width = bbox.width * 1.1
        new_height = bbox.height * 1.1

        # Calculate the new top, right, bottom, left coordinates
        top = int(center_y - new_height / 2)
        right = int(center_x + new_width / 2)
        bottom = int(center_y + new_height / 2)
        left = int(center_x - new_width / 2)

        face_bbox = (top, right, bottom, left)
        score = detection.categories[0].score
        keypoints = []
        for keypoint in detection.keypoints:
            keypoint_px = _normalized_to_pixel_coordinates(keypoint.x, keypoint.y, width, height)
            keypoints.append(keypoint_px)
        face_data.append((image_path, face_bbox, keypoints, score))

    return face_data

def process_video(video_path, model_path='detector.tflite', min_detection_confidence=0.6, generate_embeddings=True):
    detector = initialize_mediapipe_detector(model_path, min_detection_confidence)

    cap = cv2.VideoCapture(video_path)
    frame_idx = 0
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    face_data_list = []
    total_faces = 0
    with tqdm(total=total_frames // 2, desc="Processing frames") as pbar:
      while cap.isOpened():
          ret, frame = cap.read()
          if not ret:
              break

          if frame_idx % 2 == 0:
            try:
              video_path = os.path.basename(video_path)
              video_path = os.path.splitext(video_path)[0]
              frame_path = f"{video_path}_frame{frame_idx:05d}.jpg"
              cv2.imwrite(frame_path, frame)
              face_data = detect_faces_mediapipe(detector, frame_path)
              face_num = 0  # Counter for faces in the current frame
              if face_data:
                for face in face_data:
                    (image_path, (top, right, bottom, left), keypoints, score) = face
                    face_image = frame[top:bottom, left:right]
                    if face_image.size != 0:

                        face_path = f"{video_path}_frame{frame_idx:05d}_face{face_num:02d}"
                        embedding = None
                        if generate_embeddings:
                            embedding = generate_embedding(face_image)
                        face_data_list.append((face_path, (top, right, bottom, left), keypoints, score, embedding))
                        face_num += 1
              total_faces += face_num

            finally:
              os.remove(frame_path)

            pbar.set_description(f"Processing {video_path} frame {frame_idx // 2 + 1}/{total_frames // 2}")
            pbar.update(1)

          frame_idx += 1

      cap.release()
      print(total_faces)
    return face_data_list

NameError: name 'mp' is not defined

In [ ]:
#ORIGINAL MP
def process_video_and_search(video_path, emb_dict, model_path='detector.tflite', min_detection_confidence=0.6, threshold = 0.4, save_n=500):
    detector = initialize_mediapipe_detector(model_path, min_detection_confidence)

    video_name = os.path.basename(video_path)
    video_name = os.path.splitext(video_name)[0]

    output_dir = os.path.join(os.path.dirname(video_path), video_name)
    os.makedirs(output_dir, exist_ok=True)


    checkpoint_path = os.path.join(output_dir, f"{video_name}_checkpoint.pkl")
    cumulative_matches_path = os.path.join(output_dir, f"{video_name}_cumulative_matches.csv")

    if os.path.exists(checkpoint_path):
        with open(checkpoint_path, 'rb') as cp_file:
            checkpoint = pickle.load(cp_file)
            frame_idx = checkpoint['frame_idx']
            print(f"Resuming from frame {frame_idx}")
    else:
        frame_idx = 0

    if os.path.exists(cumulative_matches_path):
        cumulative_matches = pd.read_csv(cumulative_matches_path).to_dict('records')
    else:
        cumulative_matches = []

    cap = cv2.VideoCapture(video_path)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    total_faces = 0

    with tqdm(total=total_frames, initial=frame_idx, desc="Processing frames") as pbar:
      while cap.isOpened():
          ret, frame = cap.read()
          if not ret:
              break

          matches = []
          try:
              video_path = os.path.basename(video_path)
              video_path = os.path.splitext(video_path)[0]
              frame_path = f"{video_path}_frame{frame_idx:05d}.jpg"
              cv2.imwrite(frame_path, frame)
              face_data = detect_faces_mediapipe(detector, frame_path)
              face_num = 0  # Counter for faces in the current frame
              if face_data:
                #print("face " + str(face_num) + "in frame: " + str(frame_idx))
                for face in face_data:
                    (image_path, (top, right, bottom, left), keypoints, score) = face
                    face_image = frame[top:bottom, left:right]
                    if face_image is not None and face_image.size != 0:

                      vid_embedding = generate_embedding(face_image)
                      if vid_embedding is not None:
                        match_data = {
                            'name': 'NA',
                            'badge_number': 'NA',
                            'img_name': 'NA',
                            'file_name': f"{video_name}_frame{frame_idx:05d}_face{face_num:02d}",
                            'bbox_coords': (top, right, bottom, left),
                            'face_confidence_score': score,
                            'similarity': 0,
                            'matched_embedding': None
                        }
                        vid_embedding=vid_embedding[0]
                        for emb_index, emb_row in emb_dict.iterrows():
                            dict_embedding = torch.tensor(emb_row['embedding'])
                            similarity = torch.nn.functional.cosine_similarity(vid_embedding.squeeze(0), dict_embedding.squeeze(0), dim=0)
                            if similarity.item() >= threshold:
                              print(f"Match {emb_row['name']} found for frame {frame_idx} with similarity {similarity:.2f}")
                              match_data = {
                                  'name': emb_row['name'],
                                  'badge_number': emb_row['badge_number'],
                                  'img_name': emb_row['source_link'],
                                  'file_name': f"{video_name}_frame{frame_idx:05d}_face{face_num:02d}",
                                  'bbox_coords': (top, right, bottom, left),
                                  'face_confidence_score': score,
                                  'similarity': similarity.cpu().numpy(),
                                  'matched_embedding': vid_embedding
                              }
                        cumulative_matches.append(match_data)
                      face_num += 1
                total_faces += face_num

                if frame_idx % save_n == 0 and cumulative_matches:
                  pd.DataFrame(cumulative_matches).to_csv(cumulative_matches_path, index=False)
                  checkpoint = {'frame_idx': frame_idx}
                  with open(checkpoint_path, 'wb') as cp_file:
                    pickle.dump(checkpoint, cp_file)
                  print(f"Checkpoint saved at frame {frame_idx} with {len(cumulative_matches)} total matches.")

          finally:
              os.remove(frame_path)

          pbar.set_description(f"Processing {video_path} frame {frame_idx + 1}/{total_frames}")
          pbar.update(1)


          frame_idx += 1

      cap.release()
      print(f"Total mp faces detected: {total_faces}")

    # Save final cumulative results
    if cumulative_matches:
        pd.DataFrame(cumulative_matches).to_csv(cumulative_matches_path, index=False)
        print(f"Total {len(cumulative_matches)} matches saved to {cumulative_matches_path}")

    # Remove checkpoint after processing is complete
    if os.path.exists(checkpoint_path):
        os.remove(checkpoint_path)

    return cumulative_matches

In [ ]:
#@title **RETINA FACE DETECT**
!pip install -U retinaface_pytorch > /dev/null

from retinaface.pre_trained_models import get_model
from matplotlib import pyplot as plt

retfacemodel = get_model("resnet50_2020-07-20", max_size=2048)
retfacemodel.eval()

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
Downloading: "https://github.com/ternaus/retinaface/releases/download/0.01/retinaface_resnet50_2020-07-20-f168fae3c.zip" to /root/.cache/torch/hub/checkpoints/retinaface_resnet50_2020-07-20-f168fae3c.zip
100%|██████████| 96.9M/96.9M [00:02<00:00, 38.6MB/s]
/usr/local/lib/python3.10/dist-packages/torch/hub.py:665: UserWarning: Falling back to the old format < 1.6. This support will be deprecated in favor of default zipfile format introduced in 1.6. Please redo torch.save() to save it in 

In [ ]:
#test

image = cv2.imread("/content/_content_drive_MyDrive_COPY_VOL00001_NATIVES_NATIVE00001_CCRB-00001679.MOV_frame_0.jpg")
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
detections = retfacemodel.predict_jsons(image)

# Loop through detected faces
for face in detections:
    # Get bounding box
    facial_area = face['bbox']
    # Get landmarks (keypoints)
    landmarks = face['landmarks']
    # Get confidence score
    score = face['score']
    #print(f"Bounding Box: {facial_area}")
    #print(f"Keypoints: {landmarks}")
    #print(f"Confidence Score: {score}\n")

    # Draw bounding box and keypoints on the image
    cv2.rectangle(image, (facial_area[0], facial_area[1]), (facial_area[2], facial_area[3]), (0, 255, 0), 2)
    for point in landmarks:
        # Convert the coordinates to integers
        coord = (int(point[0]), int(point[1]))
        cv2.circle(image, coord, 2, (0, 0, 255), -1)

# Display the image with detected faces
plt.imshow(image)
plt.axis('off')
plt.show()

error: OpenCV(4.10.0) /io/opencv/modules/imgproc/src/color.cpp:196: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'


In [ ]:
import os
import pickle
import pandas as pd
import torch
from tqdm.notebook import tqdm
from retinaface.pre_trained_models import get_model

def detect_faces_retinaface(model, frame_path, min_detection_score):
    # Convert frame to RGB as required by the RetinaFace model
    image = cv2.imread(frame_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    detections = model.predict_jsons(image)
    face_data = []
    for face in detections:
        score = face['score']
        if score < min_detection_score:
            continue
        # Extract bounding box and landmarks
        bbox = face['bbox']
        landmarks = face['landmarks']
        score = face['score']


        face_data.append((frame_path, bbox, landmarks, score))

        #cv2.rectangle(image, (bbox[0], bbox[1]), (bbox[2], bbox[3]), (0, 255, 0), 2)


        # Display the image with detected faces
    #plt.imshow(image)
    #plt.axis('off')
    #plt.show()

    return face_data
total_face_data = []

def process_video_and_search(video_path, emb_dict, min_detection_score = 0.95, threshold=0.4, save_n=100):
    # Initialize the RetinaFace model
    retinaface_model = get_model("resnet50_2020-07-20", max_size=2048)
    retinaface_model.eval()

    video_name = os.path.basename(video_path)
    video_name = os.path.splitext(video_name)[0]

    output_dir = os.path.join(os.path.dirname(video_path), video_name)
    os.makedirs(output_dir, exist_ok=True)

    checkpoint_path = os.path.join(output_dir, f"{video_name}_checkpoint.pkl")
    cumulative_matches_path = os.path.join(output_dir, f"{video_name}_cumulative_matches.csv")

    if os.path.exists(checkpoint_path):
        with open(checkpoint_path, 'rb') as cp_file:
            checkpoint = pickle.load(cp_file)
            frame_idx = checkpoint['frame_idx']
            print(f"Resuming from frame {frame_idx}")
    else:
        frame_idx = 0

    if os.path.exists(cumulative_matches_path):
        cumulative_matches = pd.read_csv(cumulative_matches_path).to_dict('records')
    else:
        cumulative_matches = []

    cap = cv2.VideoCapture(video_path)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    total_faces = 0
    cap.set(cv2.CAP_PROP_POS_FRAMES, frame_idx)

    with tqdm(total=total_frames, initial=frame_idx, desc="Processing frames") as pbar:
        while cap.isOpened() and frame_idx < total_frames:
            ret, frame = cap.read()
            if not ret:
                break

            matches = []
            try:
                video_path = os.path.basename(video_path)
                video_path = os.path.splitext(video_path)[0]
                frame_path = f"{video_name}_frame{frame_idx:05d}.jpg"
                cv2.imwrite(frame_path, frame)

                # Use RetinaFace instead of MediaPipe
                face_data = detect_faces_retinaface(retinaface_model, frame_path, min_detection_score)
                face_num = 0  # Counter for faces in the current frame
                if face_data:
                    print("frame: " + str(frame_idx) + " faces: " + str(len(face_data)))
                    total_face_data.append(face_data)
                    for face in face_data:
                        (image_path, (left, top, right, bottom), keypoints, score) = face

                        center_x = (left + right) / 2
                        center_y = (top + bottom) / 2
                        width = right - left
                        height = bottom - top

                        new_width = width * 1.5
                        new_height = height * 1.5

                        left = int(center_x - new_width / 2)
                        right = int(center_x + new_width / 2)
                        top = int(center_y - new_height / 2)
                        bottom = int(center_y + new_height / 2)

                        left = max(0, left)
                        top = max(0, top)
                        right = min(frame.shape[1], right)
                        bottom = min(frame.shape[0], bottom)

                        face_image = frame[top:bottom, left:right]

                        match_data = {
                                    'name': 'NA',
                                    'badge_number': 'NA',
                                    'img_name': 'NA',
                                    'file_name': f"{video_name}_frame{frame_idx:05d}_face{face_num:02d}",
                                    'bbox_coords': (top, right, bottom, left),
                                    'face_confidence_score': score,
                                    'similarity': 0,
                                    'matched_embedding': None
                        }
                        if face_image is not None and face_image.size != 0:
                            vid_embedding = generate_embedding(face_image)
                            if vid_embedding is not None:
                                vid_embedding = vid_embedding[0]
                                for emb_index, emb_row in emb_dict.iterrows():
                                    dict_embedding = torch.tensor(emb_row['embedding'])
                                    similarity = torch.nn.functional.cosine_similarity(vid_embedding.squeeze(0), dict_embedding.squeeze(0), dim=0)
                                    if similarity.item() >= threshold:
                                        print(f"Match {emb_row['name']} found for frame {frame_idx} with similarity {similarity:.2f}")
                                        match_data = {
                                            'name': emb_row['name'],
                                            'badge_number': emb_row['badge_number'],
                                            'img_name': emb_row['source_link'],
                                            'file_name': f"{video_name}_frame{frame_idx:05d}_face{face_num:02d}",
                                            'bbox_coords': (top, right, bottom, left),
                                            'face_confidence_score': score,
                                            'similarity': similarity.cpu().numpy(),
                                            'matched_embedding': vid_embedding
                                        }
                            face_num += 1
                        cumulative_matches.append(match_data)
                    total_faces += face_num

                    if frame_idx % save_n == 0:
                        print("cumulative matches list: " + str(len(cumulative_matches)))
                        if cumulative_matches:
                          pd.DataFrame(cumulative_matches).to_csv(cumulative_matches_path, index=False)
                          checkpoint = {'frame_idx': frame_idx}
                          with open(checkpoint_path, 'wb') as cp_file:
                              pickle.dump(checkpoint, cp_file)
                          print(f"Checkpoint saved at frame {frame_idx} with {len(cumulative_matches)} total matches.")

            finally:
                os.remove(frame_path)

            pbar.set_description(f"Processing {video_path} frame {frame_idx + 1}/{total_frames}")
            pbar.update(1)

            frame_idx += 1

        cap.release()
        print(f"Total faces detected: {total_faces}")

    # Save final cumulative results
    if cumulative_matches:
        pd.DataFrame(cumulative_matches).to_csv(cumulative_matches_path, index=False)
        print(f"Total {len(cumulative_matches)} matches saved to {cumulative_matches_path}")

    # Remove checkpoint after processing is complete
    if os.path.exists(checkpoint_path):
        os.remove(checkpoint_path)

    return cumulative_matches


In [ ]:
video_path = "/content/drive/MyDrive/faces/success/timessquare/DEFVID_000000165.MP4"
video_similars = process_video_and_search(video_path, filtered, save_n = 100)

Processing frames:   0%|          | 0/5788 [00:00<?, ?it/s]

frame: 0 faces: 2
cumulative matches list: 2
Checkpoint saved at frame 0 with 2 total matches.
frame: 1 faces: 2
frame: 2 faces: 4
frame: 3 faces: 3
frame: 4 faces: 5
frame: 5 faces: 6
frame: 6 faces: 7
frame: 7 faces: 4
frame: 8 faces: 7
frame: 9 faces: 6
frame: 10 faces: 6
frame: 11 faces: 7
frame: 12 faces: 3
frame: 13 faces: 8
frame: 14 faces: 3
frame: 15 faces: 5
frame: 16 faces: 4
frame: 17 faces: 4
frame: 18 faces: 6
frame: 19 faces: 2
frame: 20 faces: 3
frame: 21 faces: 4
frame: 22 faces: 2
frame: 23 faces: 6
frame: 24 faces: 3
frame: 25 faces: 6
frame: 26 faces: 5
frame: 27 faces: 4
frame: 28 faces: 2
frame: 29 faces: 4
frame: 30 faces: 2
frame: 31 faces: 4
frame: 32 faces: 5
frame: 33 faces: 5
frame: 34 faces: 5
frame: 35 faces: 4
frame: 36 faces: 4
frame: 37 faces: 4
frame: 38 faces: 4
frame: 39 faces: 5
frame: 40 faces: 4
frame: 41 faces: 6
frame: 42 faces: 6
frame: 43 faces: 5
frame: 44 faces: 4
frame: 45 faces: 5
frame: 46 faces: 2
frame: 47 faces: 5
frame: 48 faces: 5
fr

In [ ]:
import pandas as pd

def remove_non_faces(csv_path, non_faces, output_csv_path=None):
    # Load the CSV file
    df = pd.read_csv(csv_path)
    print(len(df))

    # Ensure the 'badge_number' column exists
    if 'badge_number' not in df.columns:
        raise ValueError("The CSV file does not contain a 'badge_number' column.")

    # Remove rows where 'badge_number' is in the non_faces list
    df_cleaned = df[~df['badge_number'].isin(non_faces)]
    print(len(df_cleaned))

    # Optionally save the cleaned DataFrame to a new CSV file
    if output_csv_path:
        df_cleaned.to_csv(output_csv_path, index=False)

    return df_cleaned

# Example usage
csv_path = '/content/drive/MyDrive/faces/success/DEFVID_000000155/DEFVID_000000155_cumulative_matches3.csv'  # Replace with your CSV file path
#non_faces = ['#17032', '#10171', '#13404', '#16688', '#648', '#7320', '#13767', '#2126', '#5207', '#3061']

cleaned_df = remove_non_faces(csv_path, non_faces, output_csv_path='/content/drive/MyDrive/faces/success/DEFVID_000000155/DEFVID_000000155_cumulative_matches_cleaned.csv')




33183
33002


In [ ]:
import os
import glob
import cv2
import numpy as np
from PIL import Image

def extract_filename(file_name):
    # Find the position of 'frame' and 'face' in the string
    frame_index = file_name.find('_frame')
    face_index = file_name.find('_face')

    # Extract the filename up to 'frame'
    filename = file_name[:frame_index]

    # Extract the frame number between 'frame' and 'face'
    frame_number = int(file_name[frame_index + 6:face_index])

    # Extract the face number after 'face'
    face_number = int(file_name[face_index + 5:])

    return filename, frame_number, face_number


def find_video_path(videos_path, filename):
    # Use glob to find any file that matches the filename with any extension
    search_pattern = os.path.join(videos_path, filename + '.*')
    matching_files = glob.glob(search_pattern)

    if matching_files:
        # If a matching file is found, return its full path
        return matching_files[0]
    else:
        # If no file is found, return None or handle the error appropriately
        return None

In [ ]:
import pandas as pd

def concat_csvs(csv_path1, csv_path2, output_csv_path=None):
    # Read the first CSV file into a DataFrame
    df1 = pd.read_csv(csv_path1)
    len(df1)
    # Read the second CSV file into a DataFrame
    df2 = pd.read_csv(csv_path2)
    len(df2)

    # Concatenate the two DataFrames
    concatenated_df = pd.concat([df1, df2], ignore_index=True)
    len(concatenated_df)
    # If an output path is specified, save the concatenated DataFrame to a new CSV file
    if output_csv_path:
        concatenated_df.to_csv(output_csv_path, index=False)

    return concatenated_df

csv_path1 = '/content/drive/MyDrive/faces/success/DEFVID_000000155/DEFVID_000000155_cumulative_matches0.csv'
csv_path2 = '/content/drive/MyDrive/faces/success/DEFVID_000000155/DEFVID_000000155_cumulative_matches2.csv'
output_csv_path = '/content/drive/MyDrive/faces/success/DEFVID_000000155/DEFVID_000000155_cumulative_matches3.csv'

concatenated_df = concat_csvs(csv_path1, csv_path2, output_csv_path)


In [ ]:


import os
import cv2
import pandas as pd
import shutil


def draw_bounding_boxes(results_csv, videos_path, images_folder_path):
    results_df = pd.read_csv(results_csv)
    video_name = os.path.basename(results_csv).replace('_cumulative_matches.csv', '')
    csv_directory = os.path.dirname(results_csv)  # Get the directory of the input CSV file

    # Create directories for source and marked images in the CSV directory
    source_dir = os.path.join(csv_directory, 'source')
    marked_dir = os.path.join(csv_directory, 'marked')
    os.makedirs(source_dir, exist_ok=True)
    os.makedirs(marked_dir, exist_ok=True)

    # Extract frame numbers and group by them
    results_df['frame_number'] = results_df['file_name'].apply(lambda x: extract_filename(x)[1])

    grouped = results_df.groupby('frame_number')

    for frame_number, group in grouped:
        print(frame_number)
        # Since all data is for the same video, just get the video path once
        video_path = find_video_path(videos_path, video_name)

        if os.path.exists(video_path):
            cap = cv2.VideoCapture(video_path)
            if cap.isOpened():
                # Set the frame position
                cap.set(cv2.CAP_PROP_POS_FRAMES, frame_number)

                # Read the frame
                ret, frame = cap.read()
                if ret:
                    # Save the original frame with the naming convention framenumber.jpg
                    original_frame_path = os.path.join(source_dir, f"{video_name}_{frame_number}.jpg")
                    #cv2.imwrite(original_frame_path, frame)
                    original_frame_copy = frame.copy()

                    # Draw all bounding boxes for this frame
                    for _, row in group.iterrows():
                        bbox = eval(row['bbox_coords'])  # Convert string representation of tuple to actual tuple
                        img_name = row['img_name']
                        name = row['name']


                        (top, right, bottom, left) = bbox
                        #print(bbox)

                        margin = 10
                        top = max(0, top - margin)
                        left = max(0, left - margin)
                        bottom = min(frame.shape[0], bottom + margin)
                        right = min(frame.shape[1], right + margin)

                        #cropped_frame = frame[top:bottom, left:right]
                        cropped_frame = original_frame_copy[top:bottom, left:right]
                        height, width = cropped_frame.shape[:2]

                        target_width = 256
                        scaling_factor = target_width / width
                        new_height = int(height * scaling_factor)

                        scaled_cropped_frame = cv2.resize(cropped_frame, (target_width, new_height))

                        if pd.isna(img_name):
                            color = (255, 255, 0)  # Cyan for NaN
                            cropped_dir = os.path.join(csv_directory, "other")
                            os.makedirs(cropped_dir, exist_ok=True)
                            cropped_frame_path = os.path.join(cropped_dir, f"{video_name}_{frame_number}.jpg")
                            #cv2.imwrite(cropped_frame_path, cropped_frame)

                        else:
                            color = (255, 0, 255)  # YellowGreen otherwise

                            # Save the cropped image in a folder based on img_name
                            folder_name = name.split()[0][:-1]
                            cropped_dir = os.path.join(csv_directory, folder_name)
                            os.makedirs(cropped_dir, exist_ok=True)
                            img_name_sans_ext = os.path.splitext(img_name)[0]
                            cropped_frame_path = os.path.join(cropped_dir, f"{img_name_sans_ext}_{frame_number}.jpg")
                            #cv2.imwrite(cropped_frame_path, cropped_frame)

                            # Find the corresponding image by img_name and copy it if it doesn't already exist
                            img_name_path = os.path.join(images_folder_path, img_name)
                            dest_img_path = os.path.join(cropped_dir, img_name)
                            if os.path.exists(img_name_path) and not os.path.exists(dest_img_path):
                                shutil.copy(img_name_path, dest_img_path)


                            cropped_frame_with_margin = cv2.copyMakeBorder(
                                scaled_cropped_frame, margin, margin, margin, margin, cv2.BORDER_CONSTANT, value=color
                            )
                            cv2.imwrite(cropped_frame_path, cropped_frame_with_margin)

                        #cropped_frame_with_margin = cv2.copyMakeBorder(
                        #    scaled_cropped_frame, margin, margin, margin, margin, cv2.BORDER_CONSTANT, value=color
                        #)

                        #cv2.imwrite(cropped_frame_path, cropped_frame_with_margin)

                        cv2.rectangle(frame, (left, top), (right, bottom), color, 4)

                    # Save the frame with all bounding boxes in the marked directory
                    marked_frame_path = os.path.join(marked_dir, f"{video_name}_{frame_number}_marked.jpg")
                    cv2.imwrite(marked_frame_path, frame)

                else:
                    print(f"Failed to read frame: {frame_number} from video: {video_path}")
            else:
                print(f"Failed to open video: {video_path}")
            cap.release()
        else:
            print(f"Video not found: {video_path}")


In [ ]:
videos_path='/content/drive/MyDrive/faces/success'
images_folder_path = '/content/drive/MyDrive/faces/NYPD_profile_imgs'
results_csv = '/content/drive/MyDrive/faces/success/DEF_000321472/DEF_000321472_cumulative_matches.csv'

draw_bounding_boxes(results_csv, videos_path, images_folder_path)

327
328
329
606
608
1032
1468
1475
1476
1477
1478
1479
1480
1481
1482
1483
1488
2113
2117
2160
2161
2165
2192
2242
2243
2245
2246
2247
2248
2314
2315
2408
2410
2413
2414
2415
2416
2417
2418
2419
2420
2427
2428
2429
2614
2615
2729
2789
2827
2838
2868
2941
3062
3072
3087
3088
3089
3090
3091
3092
3093
3094
3097
3100
3107
3111
3112
3214
3227
3228
3230
3231
3232
3233
3234
3235
3236
3237
3238
3239
3240
3241
3242
3243
3244
3245
3246
3247
3248
3249
3250
3251
3252
3253
3254
3255
3256
3257
3258
3259
3260
3261
3262
3263
3264
3265
3266
3267
3268
3269
3270
3271
3272
3273
3274
3275
3276
3277
3278
3279
3280
3281
3282
3283
3284
3285
3286
3287
3288
3289
3290
3291
3292
3293
3294
3295
3296
3297
3298
3299
3300
3301
3303
3304
3305
3307
3311
3318
3332
3398
3461
3471
3472
3473
3474
3475
3476
3477
3478
3479
3480
3481
3482
3483
3484
3485
3486
3487
3488
3489
3490
3491
3492
3493
3494
3495
3496
3497
3498
3499
3500
3501
3502
3503
3504
3505
3506
3507
3508
3509
3510
3511
3512
3529
3530
3531
3532
3533
3534
3535
3536


In [ ]:
import cv2
import os

def replace_frames_in_video(original_video_path, marked_frames_dir, output_video_path):
    # Open the original video
    cap = cv2.VideoCapture(original_video_path)
    if not cap.isOpened():
        print(f"Error: Could not open video {original_video_path}")
        return

    # Get video properties
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    # Define the codec and create a VideoWriter object to save the new video
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

    # Load all marked frames into a dictionary keyed by frame number
    marked_frames = {}
    for file_name in os.listdir(marked_frames_dir):
        if file_name.endswith('_marked.jpg'):
            # Extract frame number from the file name
            frame_number = int(file_name.split('_')[-2])
            marked_frame_path = os.path.join(marked_frames_dir, file_name)
            marked_frame = cv2.imread(marked_frame_path)
            marked_frames[frame_number] = marked_frame

    # Process the video and replace frames
    for i in range(total_frames):
        ret, frame = cap.read()
        if not ret:
            break

        # Check if there is a marked frame for the current frame number
        if i in marked_frames:
            frame = marked_frames[i]

        # Write the frame to the output video
        out.write(frame)

    # Release resources
    cap.release()
    out.release()

    print(f"Finished processing. The video with marked frames has been saved to {output_video_path}")

# Example usage:
# replace_frames_in_video('original_video.mp4', 'path_to_marked_frames', 'output_video.mp4')


In [ ]:
original_video_path = "/content/drive/MyDrive/faces/success/timessquare/DEFVID_000000111.MP4"
marked_frames_dir = "/content/drive/MyDrive/faces/success/timessquare/DEFVID_000000111/marked"
output_video_path = "/content/drive/MyDrive/faces/success/timessquare/DEFVID_000000111/DEFVID_000000111_recomp.mp4"
replace_frames_in_video(original_video_path, marked_frames_dir, output_video_path)

Finished processing. The video with marked frames has been saved to /content/drive/MyDrive/faces/success/timessquare/DEFVID_000000111/DEFVID_000000111_recomp.mp4


In [ ]:
import os
import cv2
import pandas as pd

def draw_bounding_boxes_on_video(results_csv, videos_path, output_video_path):
    results_df = pd.read_csv(results_csv)
    video_name = os.path.basename(results_csv).replace('_cumulative_matches.csv', '')

    # Extract frame numbers and group by them
    results_df['frame_number'] = results_df['file_name'].apply(lambda x: extract_filename(x)[1])
    grouped = results_df.groupby('frame_number')

    # Find the video path
    video_path = find_video_path(videos_path, video_name)

    if os.path.exists(video_path):
        cap = cv2.VideoCapture(video_path)
        if not cap.isOpened():
            print(f"Failed to open video: {video_path}")
            return

        # Get video properties
        frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        fps = cap.get(cv2.CAP_PROP_FPS)
        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

        # Prepare the output video writer
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

        # Process each frame
        frame_number = 0
        while frame_number < total_frames:
            ret, frame = cap.read()
            if not ret:
                break

            # If there are bounding boxes for this frame, draw them
            if frame_number in grouped.groups:
                group = grouped.get_group(frame_number)
                for _, row in group.iterrows():
                    bbox = eval(row['bbox_coords'])  # Convert string representation of tuple to actual tuple
                    name = row['name']
                    (top, right, bottom, left) = bbox

                    # Add margin
                    margin = 10
                    top = max(0, top - margin)
                    left = max(0, left - margin)
                    bottom = min(frame.shape[0], bottom + margin)
                    right = min(frame.shape[1], right + margin)

                    # Choose color based on whether img_name is NaN
                    color = (255, 255, 0) if pd.isna(row['img_name']) else (255, 0, 255)

                    # Draw bounding box on the frame
                    cv2.rectangle(frame, (left, top), (right, bottom), color, 4)

            # Write the frame with bounding boxes to the output video
            out.write(frame)

            frame_number += 1

        # Release the video capture and writer objects
        cap.release()
        out.release()
        print(f"Video saved at: {output_video_path}")
    else:
        print(f"Video not found: {video_path}")


In [ ]:
def find_all_videos(input_folder):
    video_extensions = ['.mp4', '.avi', '.mov', '.m4v','.mpeg']
    video_files = []
    for root, dirs, files in os.walk(input_folder):
        for file in files:
            # Convert the file name to lowercase before checking the extension
            if any(file.lower().endswith(ext) for ext in video_extensions):
                video_files.append(os.path.join(root, file))
    return video_files



def run_directory(input_folder, emb_dict, output_dir, model_path='detector.tflite', min_detection_confidence=0.6, threshold=0.4, save_n=100):
    os.makedirs(output_dir, exist_ok=True)

    meta_checkpoint_path = os.path.join(output_dir, "directory_checkpoint.pkl")

    # Load meta checkpoint if it exists
    if os.path.exists(meta_checkpoint_path):
        with open(meta_checkpoint_path, 'rb') as meta_file:
            meta_checkpoint = pickle.load(meta_file)
            processed_videos = meta_checkpoint.get('processed_videos', [])
            long_videos = meta_checkpoint.get('long_videos', [])

            print(f"Resuming from checkpoint, {len(processed_videos)} videos already processed.")
    else:
        processed_videos = []
        long_videos = []
        meta_checkpoint = {}

    video_files = find_all_videos(input_folder)

    processed_count = 0

    with tqdm(total=len(video_files), desc="Processing videos") as pbar:
        for video_file in video_files:
          if os.path.abspath(video_file) in processed_videos or os.path.abspath(video_file) in long_videos:
                print(f"Skipping already processed video: {video_file}")
                pbar.update(1)  # Update the progress bar for skipped videos
                processed_count+=1
                print(processed_count)
                continue

          cap = cv2.VideoCapture(video_file)
          total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
          cap.release()

          if total_frames > 108000:
                print(f"Skipping long video {video_file} (total frames: {total_frames})")
                long_videos.append(os.path.abspath(video_file))  # Add skipped video to checkpoint
          else:
                print(f"Processing video: {video_file}")

                # Process the video
                face_matches =  process_video_and_search(video_file, emb_dict, output_dir, save_n = save_n)

                processed_videos.append(os.path.abspath(video_file))

          meta_checkpoint['processed_videos'] = processed_videos
          meta_checkpoint['long_videos'] = long_videos
          with open(meta_checkpoint_path, 'wb') as meta_file:
            pickle.dump(meta_checkpoint, meta_file)
          print(f"Checkpoint saved after processing video: {video_file}")

          pbar.update(1)

    print("All videos processed.")

# Example usage
input_folder = "/content/drive/MyDrive/COPY/VOL00002"
output_dir = "/content/drive/MyDrive/faces/search_output/VOL22"
run_directory(input_folder, filtered, output_dir, save_n=500)


Processing videos:   0%|          | 0/1043 [00:00<?, ?it/s]

Processing video: /content/drive/MyDrive/COPY/VOL00002/NATIVES/NATIVE00001/DEF_000271487.mp4


Processing frames:   0%|          | 0/1041 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/google/protobuf/symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


Total mp faces detected: 31
Checkpoint saved after processing video: /content/drive/MyDrive/COPY/VOL00002/NATIVES/NATIVE00001/DEF_000271487.mp4
Processing video: /content/drive/MyDrive/COPY/VOL00002/NATIVES/NATIVE00001/DEF_000271488.mp4


...

Processing frames:   0%|          | 0/275 [00:00<?, ?it/s]

Total mp faces detected: 5
Checkpoint saved after processing video: /content/drive/MyDrive/COPY/VOL00002/NATIVES/NATIVE00001/Payne-Ctrl-00000055.MP4
Processing video: /content/drive/MyDrive/COPY/VOL00002/NATIVES/NATIVE00001/Payne-Ctrl-00000064.MOV


Processing frames:   0%|          | 0/450 [00:00<?, ?it/s]

Total mp faces detected: 2
Checkpoint saved after processing video: /content/drive/MyDrive/COPY/VOL00002/NATIVES/NATIVE00001/Payne-Ctrl-00000064.MOV
Processing video: /content/drive/MyDrive/COPY/VOL00002/NATIVES/NATIVE00001/Payne-Ctrl-00000071.MP4


Processing frames:   0%|          | 0/6728 [00:00<?, ?it/s]